In [6]:
from multiprocessing.pool import ThreadPool

import numpy as np
from scipy.fftpack import fft, ifft, fftshift, ifftshift

from sknrf.device.signal import EnvelopeSignal, IndepDict

In [2]:
_fft = lambda x: fftshift(fft(x, fm_points, axis=axis)/fm_points, axes=axis)

In [3]:
x = np.random.random(2**18)
axis = -1
fm_points = x.shape[axis]

In [4]:
%time _ = list(map(_fft, [x]*4))

CPU times: user 53.5 ms, sys: 14.8 ms, total: 68.3 ms
Wall time: 67.3 ms


In [5]:
tpool = ThreadPool(4)
%time _ = list(tpool.map(_fft, [x]*4))

CPU times: user 60.6 ms, sys: 32.9 ms, total: 93.5 ms
Wall time: 45.9 ms


In [12]:
indep_map = IndepDict((("realtime", np.arange(1)),
                      ("freq", np.arange(0, 4)),
                      ("time", np.arange(0, 2**18)),
                      ("row", np.arange(0, 6)),
                      ("col", np.arange(1))))
print(indep_map.shape)
signal = EnvelopeSignal(np.random.random((1, 4, 2**18, 6, 1)) + 1j*np.random.random((1, 4, 2**18, 6, 1)), indep_map=indep_map)
axis = -3
fm_points = signal.shape[axis]
fm_points
signal.dtype
signal.flags

(1, 4, 262144, 6, 1)


  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [17]:
split_signal = np.split(np.asarray(signal), signal.shape[axis - 1], axis=axis - 1)
%time fs_signal = np.concatenate(list(map(_fft, split_signal)), axis=axis - 1)

CPU times: user 493 ms, sys: 95.6 ms, total: 588 ms
Wall time: 585 ms


In [18]:
tpool = ThreadPool(2)
split_signal = np.split(np.asarray(signal), signal.shape[axis - 1], axis=axis - 1)
%time fs_signal = np.concatenate(list(tpool.map(_fft, split_signal)), axis=axis - 1)

CPU times: user 558 ms, sys: 113 ms, total: 671 ms
Wall time: 503 ms


In [16]:
fs_signal.size

6291456